In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools
import random

In [62]:
import torch
import torch.nn as nn

In [63]:
device = 'cuda'

# 313 881


In [4]:
pos_weight = 881/313

In [5]:
class BertModel(nn.Module):
    def __init__(self, voc_size:int=30000, seq_len: int=512, d_model: int=384, d_ff:int=3072, pad_idx: int=1,
                num_encoder: int=12, num_heads: int=12, n_layers=6, dropout: float=0.1):
        super(BertModel, self).__init__()
        self.pad_idx = pad_idx
        self.layers = nn.ModuleList([nn.TransformerEncoderLayer(d_model=d_model, nhead=8, batch_first=True) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 1)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        output = input
        for layer in self.layers:
            output = layer(output)
        #output = output.mean(dim=1, keepdim=True)
        output = output[:,-1]
        h_pooled = self.activ1(self.fc(output)) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]
        return logits_clsf # [B, S, D_model]

In [6]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    
    ## Method 2 using ravel()
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    n=10
    preds = [round(i) for i in preds.ravel()]
    acc = metrics.accuracy_score(labels.ravel(), preds)
    f1 = metrics.f1_score(labels.ravel(), preds)
    preds = [preds[i*n:(i+1)*n] for i in range(len(preds)//6)]
    #target_names = list(genre_list.keys())
    #print(classification_report(labels,preds, target_names=target_names))
    return {"auc_micro": auc_micro, "acc" : acc, "f1" : f1, 'pred' : preds}

In [93]:
def train_one_epoch(dataloader, model, optimizer, device, loss_fn):
    model.train()
    tk0 = tqdm(dataloader, total=len(dataloader))
    total_loss = 0.0
    
    for source_tensor, label in tk0:
        optimizer.zero_grad()
        output = model(source_tensor.to(device))
        loss = loss_fn(output.view(-1, output.size(-1)), torch.FloatTensor([label]).to(device))
        
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(dataloader) 
    print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def test_one_epoch(testset, model, loss_fn):
    model.eval()
    tk0 = tqdm(testset, total=len(testset))
    targets = []
    outputs = []
    total_loss = 0.0
    with torch.no_grad():
        for source_tensor, label in tk0:
            label = torch.FloatTensor([label])
            output = model(source_tensor.to(device))
            loss = loss_fn(output.view(-1, output.size(-1)), label.to(device))
            total_loss += loss
            targets.extend(label)
            outputs.extend(torch.sigmoid(output[0]).to('cpu'))
    avg_valid_loss = total_loss / len(testset) 
    score = log_metrics(outputs, targets)
    print(" Average valid loss: {0:.2f}".format(avg_valid_loss))  
    print('AUC_SCORE: ', score["auc_micro"], " acc: ", score["acc"], "f1: ", score["f1"])
    return score, avg_valid_loss
    
def fit(model, train_dataloader, valid_dataloader=None,test_dataloader=None, EPOCHS=3, lr=0.0000005):
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight)) #ignore padding
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, model, optimizer, device, loss_fn)
        auc, loss = test_one_epoch(valid_dataloader,model, loss_fn)
        if i == 0:
            torch.save(model,'rnn_best.model')
            best_f1 = auc['f1']
        if  best_f1<auc['f1']:
            best_f1 = auc['f1']
            print(i,' EPOCH BEST MODEL!')
            torch.save(model,'rnn_best.model')
        print('Test')
        auc, loss = test_one_epoch(test_dataloader,model, loss_fn)
    best_model = torch.load('rnn_best.model')
    auc, loss  = test_one_epoch(test_dataloader,best_model, loss_fn)
    print('BEST MODEL')
    print('AUC_SCORE: ', auc["auc_micro"], " acc: ", auc["acc"], "f1: ", auc["f1"])
    return auc

            best_f1 = auc['f1']
        if  auc['f1']>best_f1:
            best_f1 = auc['f1']

In [94]:
import pickle
with open('movie_split_independent film.pickle','rb') as fw:
    movie_split = pickle.load(fw)

In [95]:
path1 = 'word2vec_model_edge/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,400)
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
                    
                    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])

In [96]:
bert = BertModel(d_model=400).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 117.49it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 793.86it/s]

 Average valid loss: 0.96
AUC_SCORE:  0.44291497975708505  acc:  0.7768595041322314 f1:  0.0


...

 Average valid loss: 1.03
AUC_SCORE:  0.5493927125506073  acc:  0.6528925619834711 f1:  0.34375
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 828.28it/s]


 Average valid loss: 0.95
AUC_SCORE:  0.6219618055555556  acc:  0.5833333333333334 f1:  0.358974358974359


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 771.70it/s]

 Average valid loss: 0.92
AUC_SCORE:  0.6037326388888888  acc:  0.625 f1:  0.3835616438356164
BEST MODEL
AUC_SCORE:  0.6037326388888888  acc:  0.625 f1:  0.3835616438356164
{'auc_micro': 0.6037326388888888, 'acc': 0.625, 'f1': 0.3835616438356164, 'pred': [[0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 0, 1, 1, 0, 1, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 1, 1, 0, 0], [], [], [], [], [], [], [], []]}


In [97]:
random.randint(0,100)

44

In [98]:
import random

# SUM

In [99]:
import pickle
with open('emb_sum.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [100]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
    if movie_name not in movie_emb.keys():
        continue
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [101]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

953
121
118


In [102]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 953/953 [00:07<00:00, 119.83it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 864.18it/s]


 Average valid loss: 0.94
AUC_SCORE:  0.6149797570850203  acc:  0.7851239669421488 f1:  0.0
Test


...

 Average valid loss: 0.93
AUC_SCORE:  0.6307692307692309  acc:  0.6363636363636364 f1:  0.4054054054054054
Test


100%|████████████████████████████████████████| 118/118 [00:00<00:00, 890.00it/s]


 Average valid loss: 0.94
AUC_SCORE:  0.5862700228832952  acc:  0.5932203389830508 f1:  0.3513513513513513


100%|████████████████████████████████████████| 118/118 [00:00<00:00, 887.02it/s]

 Average valid loss: 0.90
AUC_SCORE:  0.6013729977116705  acc:  0.6186440677966102 f1:  0.2857142857142857
BEST MODEL
AUC_SCORE:  0.6013729977116705  acc:  0.6186440677966102 f1:  0.2857142857142857
{'auc_micro': 0.6013729977116705, 'acc': 0.6186440677966102, 'f1': 0.2857142857142857, 'pred': [[0, 0, 0, 0, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 0, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1], [], [], [], [], [], [], []]}


# Doc2vec

In [103]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [104]:
path1 = 'doc2vec/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Doc2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.dv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.dv.index_to_key:
            emb = w2v.dv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [105]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:07<00:00, 122.13it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 923.84it/s]


 Average valid loss: 0.96
AUC_SCORE:  0.46842105263157896  acc:  0.7851239669421488 f1:  0.07142857142857144
Test


...

 Average valid loss: 1.12
AUC_SCORE:  0.4724696356275303  acc:  0.5619834710743802 f1:  0.32911392405063294
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 855.74it/s]


 Average valid loss: 1.07
AUC_SCORE:  0.4835069444444444  acc:  0.5666666666666667 f1:  0.23529411764705885


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 845.57it/s]

 Average valid loss: 0.99
AUC_SCORE:  0.49175347222222215  acc:  0.6583333333333333 f1:  0.2545454545454545
BEST MODEL
AUC_SCORE:  0.49175347222222215  acc:  0.6583333333333333 f1:  0.2545454545454545
{'auc_micro': 0.49175347222222215, 'acc': 0.6583333333333333, 'f1': 0.2545454545454545, 'pred': [[0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [], [], [], [], [], [], [], []]}


# 평균 임베딩

In [106]:
import pickle
with open('emb_avg2.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [107]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [108]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
121
120


In [109]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 114.81it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 754.49it/s]

 Average valid loss: 0.96
AUC_SCORE:  0.4291497975708502  acc:  0.7851239669421488 f1:  0.0


...

 Average valid loss: 1.00
AUC_SCORE:  0.5797570850202429  acc:  0.6033057851239669 f1:  0.35135135135135137
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 811.12it/s]


 Average valid loss: 0.97
AUC_SCORE:  0.6184895833333333  acc:  0.6166666666666667 f1:  0.3947368421052631


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 821.07it/s]

 Average valid loss: 0.96
AUC_SCORE:  0.615017361111111  acc:  0.6083333333333333 f1:  0.37333333333333335
BEST MODEL
AUC_SCORE:  0.615017361111111  acc:  0.6083333333333333 f1:  0.37333333333333335
{'auc_micro': 0.615017361111111, 'acc': 0.6083333333333333, 'f1': 0.37333333333333335, 'pred': [[0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 1, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 1, 1, 1, 1, 0], [0, 0, 1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 1, 1, 0, 0], [], [], [], [], [], [], [], []]}


# 그냥 metapath

In [110]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [111]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
121
120


In [112]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 118.70it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 828.97it/s]

 Average valid loss: 0.96
AUC_SCORE:  0.4279352226720648  acc:  0.7851239669421488 f1:  0.0


...

 Average valid loss: 1.06
AUC_SCORE:  0.520242914979757  acc:  0.6198347107438017 f1:  0.28125
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 853.55it/s]


 Average valid loss: 0.92
AUC_SCORE:  0.6128472222222222  acc:  0.65 f1:  0.36363636363636365


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 857.62it/s]

 Average valid loss: 0.91
AUC_SCORE:  0.6137152777777778  acc:  0.65 f1:  0.36363636363636365
BEST MODEL
AUC_SCORE:  0.6137152777777778  acc:  0.65 f1:  0.36363636363636365
{'auc_micro': 0.6137152777777778, 'acc': 0.65, 'f1': 0.36363636363636365, 'pred': [[0, 1, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 1, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 1, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 1, 1, 0, 0], [], [], [], [], [], [], [], []]}


# Main_act

In [113]:
path1 = 'word2vec_model_extra+emb/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [114]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 119.71it/s]


 Average training loss: 0.96


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 839.95it/s]


 Average valid loss: 0.95
AUC_SCORE:  0.4732793522267207  acc:  0.7851239669421488 f1:  0.0
Test


...

 Average valid loss: 1.05
AUC_SCORE:  0.5336032388663968  acc:  0.6115702479338843 f1:  0.22950819672131148
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 854.40it/s]


 Average valid loss: 0.97
AUC_SCORE:  0.5850694444444444  acc:  0.6166666666666667 f1:  0.3428571428571428


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 849.21it/s]

 Average valid loss: 0.93
AUC_SCORE:  0.5711805555555556  acc:  0.6166666666666667 f1:  0.3428571428571428
BEST MODEL
AUC_SCORE:  0.5711805555555556  acc:  0.6166666666666667 f1:  0.3428571428571428
{'auc_micro': 0.5711805555555556, 'acc': 0.6166666666666667, 'f1': 0.3428571428571428, 'pred': [[0, 1, 0, 1, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 1, 1, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 1, 1, 1, 0, 1], [], [], [], [], [], [], [], []]}


# pos

In [115]:
path1 = 'word2vec_model_extra+pos/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,400)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [116]:
bert2 = BertModel(d_model=400).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=25)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 962/962 [00:08<00:00, 117.23it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 815.50it/s]


 Average valid loss: 0.95
AUC_SCORE:  0.40968421052631576  acc:  0.7916666666666666 f1:  0.0
Test


...

 Average valid loss: 1.03
AUC_SCORE:  0.47326315789473683  acc:  0.5916666666666667 f1:  0.2222222222222222
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 790.62it/s]


 Average valid loss: 0.92
AUC_SCORE:  0.5993923611111112  acc:  0.6166666666666667 f1:  0.36111111111111105


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 776.68it/s]

 Average valid loss: 0.91
AUC_SCORE:  0.5907118055555556  acc:  0.6166666666666667 f1:  0.2580645161290323
BEST MODEL
AUC_SCORE:  0.5907118055555556  acc:  0.6166666666666667 f1:  0.2580645161290323
{'auc_micro': 0.5907118055555556, 'acc': 0.6166666666666667, 'f1': 0.2580645161290323, 'pred': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 0, 0], [], [], [], [], [], [], [], []]}


# Text rank

In [117]:
path1 = 'word2vec_model_text/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [118]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 117.53it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 821.01it/s]


 Average valid loss: 0.94
AUC_SCORE:  0.5850202429149798  acc:  0.7851239669421488 f1:  0.0
Test


...

 Average valid loss: 1.04
AUC_SCORE:  0.5279352226720648  acc:  0.6198347107438017 f1:  0.2580645161290323
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 835.97it/s]


 Average valid loss: 0.97
AUC_SCORE:  0.5646701388888888  acc:  0.5833333333333334 f1:  0.2857142857142857


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 684.22it/s]

 Average valid loss: 0.92
AUC_SCORE:  0.5551215277777778  acc:  0.6333333333333333 f1:  0.2413793103448276
BEST MODEL
AUC_SCORE:  0.5551215277777778  acc:  0.6333333333333333 f1:  0.2413793103448276
{'auc_micro': 0.5551215277777778, 'acc': 0.6333333333333333, 'f1': 0.2413793103448276, 'pred': [[0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [], [], [], [], [], [], [], []]}


In [119]:
path1 = 'word2vec_model_text2/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [120]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 117.81it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 833.01it/s]

 Average valid loss: 0.94
AUC_SCORE:  0.6218623481781377  acc:  0.7768595041322314 f1:  0.06896551724137931


...

 Average valid loss: 1.03
AUC_SCORE:  0.5635627530364372  acc:  0.6859504132231405 f1:  0.32142857142857145
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 871.81it/s]


 Average valid loss: 0.99
AUC_SCORE:  0.5638020833333333  acc:  0.6166666666666667 f1:  0.3428571428571428


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 850.85it/s]


 Average valid loss: 0.92
AUC_SCORE:  0.56640625  acc:  0.6333333333333333 f1:  0.2413793103448276
BEST MODEL
AUC_SCORE:  0.56640625  acc:  0.6333333333333333 f1:  0.2413793103448276
{'auc_micro': 0.56640625, 'acc': 0.6333333333333333, 'f1': 0.2413793103448276, 'pred': [[0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [], [], [], [], [], [], [], []]}


In [121]:
path1 = 'word2vec_model_text3/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'independent film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [122]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 119.58it/s]


 Average training loss: 0.95


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 861.54it/s]

 Average valid loss: 0.96
AUC_SCORE:  0.3777327935222672  acc:  0.7851239669421488 f1:  0.0


...

 Average valid loss: 1.14
AUC_SCORE:  0.4761133603238867  acc:  0.6528925619834711 f1:  0.29999999999999993
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 852.26it/s]


 Average valid loss: 0.96
AUC_SCORE:  0.5733506944444444  acc:  0.5666666666666667 f1:  0.2571428571428572


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 818.85it/s]

 Average valid loss: 0.95
AUC_SCORE:  0.5763888888888888  acc:  0.55 f1:  0.2702702702702703
BEST MODEL
AUC_SCORE:  0.5763888888888888  acc:  0.55 f1:  0.2702702702702703
{'auc_micro': 0.5763888888888888, 'acc': 0.55, 'f1': 0.2702702702702703, 'pred': [[0, 0, 0, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 0, 0, 1, 0, 1, 1, 1, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 1, 1, 0, 1], [], [], [], [], [], [], [], []]}


# 거리로 분석

In [156]:
path1 = 'word2vec_model_dim=384/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1

    ans = torch.zeros(1,1,512)
    for i in range(scene_num):
        if i ==0:
            continue
        #ans[0,0,i] += util.cos_sim(embs[0][i],embs[0][i-1])[0][0]
        ans[0,0,i] += torch.cdist(embs[0][i].unsqueeze(0),embs[0][i-1].unsqueeze(0))[0][0]
        
    embs = ans
    
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [157]:
bert2 = BertModel(d_model=512).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 929/929 [00:07<00:00, 127.24it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 980.24it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.5272873194221509  acc:  0.4700854700854701 f1:  0.3404255319148936
EPOCHS:2
TRAIN


...

 Average training loss: 1.02


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 989.84it/s]


 Average valid loss: 1.13
AUC_SCORE:  0.4562600321027287  acc:  0.47863247863247865 f1:  0.2823529411764706


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 985.66it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
BEST MODEL
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
{'auc_micro': 0.5497592295345104, 'acc': 0.41025641025641024, 'f1': 0.3551401869158879, 'pred': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 1, 0, 0, 1, 1], [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0], [], [], [], [], [], [], []]}
